In [3]:
using Pkg
Pkg.add("Plots")
Pkg.update(io=devnull)
using Plots
using Plots.PlotMeasures

   Resolving package versions...
   Installed JpegTurbo_jll ──────────────── v3.1.4+0
   Installed libfdk_aac_jll ─────────────── v2.0.4+0
   Installed Libmount_jll ───────────────── v2.41.2+0
   Installed x265_jll ───────────────────── v4.1.0+0
   Installed LERC_jll ───────────────────── v4.0.1+0
   Installed GR_jll ─────────────────────── v0.73.21+0
   Installed libdecor_jll ───────────────── v0.2.2+0
   Installed LoggingExtras ──────────────── v1.2.0
   Installed Opus_jll ───────────────────── v1.6.0+0
   Installed Xorg_xkbcomp_jll ───────────── v1.4.7+0
   Installed GR ─────────────────────────── v0.73.21
   Installed Measures ───────────────────── v0.3.3
   Installed ConcurrentUtilities ────────── v2.5.0
   Installed Grisu ──────────────────────── v1.0.2
   Installed Contour ────────────────────── v0.6.3
   Installed Xorg_xcb_util_wm_jll ───────── v0.4.2+0
   Installed Xorg_xcb_util_image_jll ────── v0.4.1+0
   Installed RecipesPipeline ────────────── v0.6.12
   Installed Xorg_lib

In [ ]:
struct Result
    net :: String
    spec :: String
    status :: String
    runtime :: Int64
    num_propagation :: Int64
    num_input_splits :: Int64
    num_neuron_splits :: Int64
    initial_δ_bound :: Float64
    final_δ_bound :: Float64
end

struct Results
    verifier_name :: String
    results :: Vector{Result}
end

In [ ]:
function read_results(name::String, csv_dir::String; sort_func=nothing)
    results = Result[]
    open(csv_dir, "r") do f
        while !eof(f)
            line = split(readline(f), ",")
            net = replace(line[1], " " => "")
            spec = replace(line[2], " " => "")
            status = replace(line[3], " " => "")
            runtime = parse(Int64, string(line[4]))
            num_propagation = parse(Int64, string(line[5]))
            num_input_splits = parse(Int64, string(line[6]))
            num_neuron_splits = parse(Int64, string(line[7]))
            initial_δ_bound = parse(Float64, string(line[8]))
            final_δ_bound = parse(Float64, string(line[9]))
            push!(results, Result(net, spec, status, runtime, num_propagation, num_input_splits, num_neuron_splits, initial_δ_bound, final_δ_bound))
        end
    end
    if !isnothing(sort_func)
        sort!(results, by=sort_func)
    end
    return Results(name, results)
end

In [ ]:
cur_dir = @__DIR__
exp_final_dir = joinpath(cur_dir, "experiments_final")

acas_name = "acas-0.5"
mnist_name = "mnist-1.0"

results_name = "results.csv"

verydiff_name = "VeryDiff"
verydiff_acas_path = joinpath(exp_final_dir, verydiff_name, acas_name, results_name)
verydiff_mnist_path = joinpath(exp_final_dir, verydiff_name, mnist_name, results_name)

# heuristic_variants = [["ZB" "ZU" "DB" "DU"], ["Alt" "Base" "Input" "DiffZono" "Input-DiffZono"]]
heuristic_variants = [["ZU" "DU"], ["Base" "Input-DiffZono"]]

names = ["DeepSplit", vcat.("ZonoContract", ["", "Inter", "Pre", "Post"])]
for v in heuristic_variants
    names = reduce(vcat, [d * "-" .* v for d in names])
end

acas_paths = Tuple{String, String}[]
mnist_paths = Tuple{String, String}[]
for name in names
    dir = joinpath(exp_final_dir, name)
    push!(acas_paths, (name, joinpath(dir, acas_name, results_name)))
    # push!(mnist_paths, (name, joinpath(dir, mnist_name, results_name)))
end

push!(acas_paths, (verydiff_name, verydiff_acas_path))
# push!(mnist_paths, (verydiff_name, verydiff_mnist_path))

println("...")

In [ ]:
sort_func_solved = r -> r.runtime
sort_func_unsolved = r -> (r.initial_δ_bound - r.final_δ_bound) / (r.initial_δ_bound - 0.5)

acas_results = [read_results(n, p) for (n, p) in acas_paths]
# mnist_results = [read_results(n, p; sort_func=sort_func) for (n, p) in mnist_paths]

acas_solved, acas_unsolved = Results[], Results[]
# mnist_solved, mnist_unsolved = Results[], Results[]

for results in acas_results
    mask = map(r -> r.status != "UNKNOWN", results.results)
    solved = results.results[mask]
    unsolved = results.results[.!mask]
    sort!(solved, by=sort_func_solved)
    sort!(unsolved, by=sort_func_unsolved, rev=true)
    push!(acas_solved, Results(results.verifier_name, solved))
    push!(acas_unsolved, Results(results.verifier_name, unsolved))
end

# for res in acas_unsolved
#     for inst in res.results
#         @assert inst.runtime / 1e9 >= 120.0
#     end
# end

# for results in mnist_results
    # mask = map(r -> r.status != "UNKNOWN", results.results)
    # solved = results.results[mask]
    # unsolved = results.results[.!mask]
    # sort!(solved, by=sort_func_solved)
    # sort!(solved, by=sort_func_unsolved)
    # push!(mnist_solved, Results(results.verifier_name, solved))
    # push!(mnist_unsolved, Results(results.verifier_name, unsolved))
# end


In [ ]:
x_acas_solved = map(r -> 1:size(r.results, 1), acas_solved)
y_acas_solved = map(acas_solved) do r
    return map(r.results) do r
        return r.runtime / 1e9
    end
end

# x_mnist_solved = map(r -> 1:size(r.results, 1), mnist_solved)
# y_mnist_solved = map(mnist_solved) do r
#     return map(r.results) do r
#         return r.runtime / 1e9
#     end 
# end


In [ ]:
verydiff_rgx = r"VeryDiff"
deepsplit_rgx = r"DeepSplit"
zonocontract_rgx = r"ZonoContract$"
zonocontract_pre_rgx = r"ZonoContractPre"
zonocontract_post_rgx = r"ZonoContractPost"
zonocontract_inter_rgx = r"ZonoContractInter"
alt_rgx = r"((ZB|ZU|DB|DU)\-Alt$)"
base_rgx = r"((ZB|ZU|DB|DU)\-Base$)"
input_rgx = r"((ZB|ZU|DB|DU)\-Input$)"
diffzono_rgx = r"((ZB|ZU|DB|DU)\-DiffZono$)"
input_diffzono_rgx = r"((ZB|ZU|DB|DU)\-Input\-DiffZono$)"

config_match_rgx = (res, rgx) -> !isnothing(findfirst(rgx, res.verifier_name))
config_match = (res, rgxs) -> all(map(rgx -> config_match_rgx(res, rgx), rgxs))
filter_indices = (res, rgxs) -> filter(i -> config_match(res[i], rgxs), 1:size(res, 1))

# acas_indices = rgxs -> filter_indices(acas_solved, rgxs)
# mnist_indices = rgxs -> filter_indices(mnist_solved, rgxs)

In [ ]:
verydiff_acas_results = acas_results[filter_indices(acas_results, [verydiff_rgx])][1]

for res in acas_results
    if res.verifier_name == verydiff_acas_results.verifier_name
        continue
    end
    for i in 1:100
        s₁, s₂ = verydiff_acas_results.results[i].status, res.results[i].status
        p₁, p₂ = verydiff_acas_results.results[i].spec, res.results[i].spec
        @assert s₁ == s₂ || s₁ == "UNKNOWN" || s₂ == "UNKNOWN" "(status₁, prop₁): ($s₁, $p₁), (status₂, prop₂): ($s₂, $p₂)"
    end
end


In [ ]:
verydiff_config = ("VeryDiff", [verydiff_rgx])

base_configs = [
    ("LP-Base", [deepsplit_rgx, base_rgx]), 
    ("ZC-Base", [zonocontract_rgx, base_rgx]),
    ("ZC-Pre-Base", [zonocontract_pre_rgx, base_rgx]),
    ("ZC-Post-Base", [zonocontract_post_rgx, base_rgx]),
    ("ZC-Inter-Base", [zonocontract_inter_rgx, base_rgx]),
]

input_diffzono_configs = [
    ("LP-Input-DiffZono", [deepsplit_rgx, input_diffzono_rgx]),
    ("ZC-Input-DiffZono", [zonocontract_rgx, input_diffzono_rgx]),
    ("ZC-Pre-Input-DiffZono", [zonocontract_pre_rgx, input_diffzono_rgx]),
    ("ZC-Post-Input-DiffZono", [zonocontract_post_rgx, input_diffzono_rgx]),
    ("ZC-Inter-Input-DiffZono", [zonocontract_inter_rgx, input_diffzono_rgx]),
]


In [ ]:
num_configs = size(base_configs, 1) + 1

p = plot(dpi=500, xlims=(0, 100), ylims=(0, 120), widen=true, xguidefontsize=5, yguidefontsize=5, legend=:legend; layout=num_configs)
xlabel!("Number of Instances Verified")
ylabel!("Time (s)")

name, rgxs = verydiff_config
indices = filter_indices(acas_solved, rgxs)
plot!(p, x_acas_solved[indices], y_acas_solved[indices], title=name, label=nothing, subplot=3)

for (i, (name, rgxs)) in enumerate(base_configs)
    indices = filter_indices(acas_solved, rgxs)
    plot!(p, x_acas_solved[indices], y_acas_solved[indices], title=name, label=["ZU" "DU"], subplot=(i + i ÷ 3))
end

plot!(p, plot_title="ACAS Benchmarks", plot_titlevspan=0.1)

savefig(p, "acas_benchmarks_base.png")

p

In [ ]:
num_configs = size(input_diffzono_configs, 1) + 1

p = plot(dpi=500, xlims=(0, 100), ylims=(0, 120), widen=true, xguidefontsize=5, yguidefontsize=5, legend=:legend; layout=num_configs)
xlabel!("Number of Instances Verified")
ylabel!("Time (s)")

name, rgxs = verydiff_config
indices = filter_indices(acas_solved, rgxs)
plot!(p, x_acas_solved[indices], y_acas_solved[indices], title=name, label=nothing, subplot=3)

for (i, (name, rgxs)) in enumerate(input_diffzono_configs)
    indices = filter_indices(acas_solved, rgxs)
    plot!(p, x_acas_solved[indices], y_acas_solved[indices], title=name, label=["ZU" "DU"], subplot=(i + i ÷ 3))
end

plot!(p, plot_title="ACAS Benchmarks", plot_titlevspan=0.1)

savefig(p, "acas_benchmarks_input_diffzono.png")

p

In [ ]:
x_acas_unsolved = map(r -> 1:size(r.results, 1), acas_unsolved)
y_acas_unsolved = map(acas_unsolved) do r
    return map(r.results) do r
        return (r.initial_δ_bound - r.final_δ_bound) / (r.initial_δ_bound - 0.5)
    end
end

# x_mnist_unsolved = map(r -> 1:size(r.results, 1), mnist_unsolved)
# y_mnist_unsolved = map(mnist_unsolved) do r
#     return map(r.results) do r
        # return (r.initial_δ_bound - r.final_δ_bound) / (r.initial_δ_bound - 0.5)
#     end 
# end

In [ ]:
num_configs = size(base_configs, 1) + 1

p = plot(dpi=500, xlims=(0, 100), ylims=(0, 1), widen=true, xguidefontsize=5, yguidefontsize=5; layout=num_configs)
xlabel!("Number of Instances Unsolved")
ylabel!("Improvement (%)")

name, rgxs = verydiff_config
indices = filter_indices(acas_unsolved, rgxs)
plot!(p, x_acas_unsolved[indices], y_acas_unsolved[indices], title=name, label=nothing, subplot=3)

for (i, (name, rgxs)) in enumerate(base_configs)
    indices = filter_indices(acas_unsolved, rgxs)
    plot!(p, x_acas_unsolved[indices], y_acas_unsolved[indices], title=name, label=["ZU" "DU"], subplot=(i + i ÷ 3))
end

plot!(p, plot_title="ACAS Benchmarks", plot_titlevspan=0.1)

savefig(p, "acas_benchmarks_unsolved_base.png")

p

In [ ]:
num_configs = size(input_diffzono_configs, 1) + 1

p = plot(dpi=500, xlims=(0, 100), ylims=(0, 1), widen=true, xguidefontsize=5, yguidefontsize=5; layout=num_configs)
xlabel!("Number of Instances Unsolved")
ylabel!("Improvement (%)")

name, rgxs = verydiff_config
indices = filter_indices(acas_unsolved, rgxs)
plot!(p, x_acas_unsolved[indices], y_acas_unsolved[indices], title=name, label=nothing, subplot=3)

for (i, (name, rgxs)) in enumerate(input_diffzono_configs)
    indices = filter_indices(acas_unsolved, rgxs)
    plot!(p, x_acas_unsolved[indices], y_acas_unsolved[indices], title=name, label=["ZU" "DU"], subplot=(i + i ÷ 3))
end

plot!(p, plot_title="ACAS Benchmarks", plot_titlevspan=0.1)

savefig(p, "acas_benchmarks_unsolved_input_diffzono.png")

p